## REMO CMOR test

In [1]:
import xarray as xr
#import cf_xarray as cfxr
import cftime
import pandas as pd
#from datetime import datetime as dt
from datetime import timedelta as td
import pyremo
import cordex as cx
import xclim as xc

import glob
import sys, os
import numpy as np
#from urllib.request import urlopen
import json
import cmor

#import iteration_utilities
#from pyhomogenize.time_control import time_control

could not find pyintorg, you need this for preprocessing. Please consider installing it from https://git.gerics.de/python/pyintorg.git


### Define relevant functions

In [2]:
def open_mfdataset(files,
                   use_cftime=True,
                   parallel=True,
                   data_vars='minimal',
                   chunks={'time':1}, 
                   coords='minimal',
                   compat='override',
                   drop=None,
                   **kwargs):
    """optimized function for opening large cf datasets.

    based on https://github.com/pydata/xarray/issues/1385#issuecomment-561920115
    
    """
    def drop_all_coords(ds):
        return ds.reset_coords(drop=True)
    
    ds = xr.open_mfdataset(files,
                           parallel=parallel,
                           decode_times=False,
                           combine='by_coords',
                           preprocess=drop_all_coords,
                           decode_cf=False,
                           chunks=chunks,
                           data_vars=data_vars,
                           coords='minimal',
                           compat='override',
                           **kwargs)
    return xr.decode_cf(ds, use_cftime=use_cftime)


def correct_monthly_time_axis(ds):

    def _begin_of(date, precision='day'):
        while True:
            m  = getattr(date, precision)
            pm = getattr((date - td(hours=1)), precision)
            if pm !=m:
                return date
            date -= td(hours=1)
        
    def _end_of(date, precision='day'):
        while True:
            m  = getattr(date, precision)
            nm = getattr((date + td(hours=1)), precision)
            if nm != m:
                return date
            date += td(hours=1)
            
    def check_int(int):
        if int == 0:
            return 1
        return int
        
    def _year(str):
        return check_int(int(str[:4]))
    
    def _month(str):
        return check_int(int(str[4:6]))
    
    def _day(str):
        return check_int(int(str[6:8]))
    
    def numpy_to_date(da):
        numpyfloat = str(da.values)
        year  = _year(numpyfloat)
        month = _month(numpyfloat)
        day   = _day(numpyfloat)
        return cftime.datetime(year, month, day, calendar=da.calendar)
    
    time = [numpy_to_date(t) for t in ds.time]
    start = _begin_of(time[0], 'month')
    end   = _end_of(time[-1], 'month')
    t1    = xr.cftime_range(start, end, freq='MS')
    t2    = xr.cftime_range(start, end, freq='M')
    refdate = cftime.datetime(1850, 1, 1, calendar = ds.time.calendar)
    dates = t1 + (t2 - t1 + td(days=1)) / 2
    return (dates-refdate)/ np.timedelta64(1, 'D')
    
def select_domain(ds, domain):
    domain = cx.cordex_domain(domain)
    return ds.sel(rlon=domain.rlon, rlat=domain.rlat, method='nearest')


def add_bnds(ds, bnds):
    if isinstance(bnds, str): bnds = [bnds]
    ds_bnds = ds.cf.add_bounds(bnds)
    for bnd in bnds:
        ds_bnds = ds_bnds.rename({bnd+'_bounds':bnd+'_bnds'})
    ds_bnds = ds_bnds.rename_dims({'bounds' : 'bnds'})
    return ds_bnds


def get_bnds(da):
    values = da.values
    bnds = [None]*(len(values) + 1)
    bnds[0] = values[0] - (values[1] - values[0]) / 2
    bnds[len(values)] = values[-1] + (values[-1] - values[-2]) / 2
    i = 1
    while i < len(values):
        bnds[i] = values[i] - (values[i] - values[i-1]) / 2
        i += 1
    return bnds  

def create_dataset(old_ds, var=None, table=pd.DataFrame()):
    if not var or table.empty: return ds
    new_ds = xr.Dataset(data_vars = {var : ds[var_table.variable.values[0]],
                                     'rotated_latitude_longitude' : ds.rotated_latitude_longitude}, 
                        attrs = ds.attrs)
    new_ds[var] = xc.units.convert_units_to(new_ds[var], var_table.units.values[0])
    new_ds[var].attrs["units"] = var_table.units.values[0]
    return new_ds

### Read REMO raw output data

In [3]:
pattern = '/work/ch0636/g300046/remo_results_056000/1979/e056000m*.nc'
filenames = glob.glob(pattern)
filenames.sort()
filenames

['/work/ch0636/g300046/remo_results_056000/1979/e056000m197901.nc',
 '/work/ch0636/g300046/remo_results_056000/1979/e056000m197902.nc',
 '/work/ch0636/g300046/remo_results_056000/1979/e056000m197903.nc',
 '/work/ch0636/g300046/remo_results_056000/1979/e056000m197904.nc',
 '/work/ch0636/g300046/remo_results_056000/1979/e056000m197905.nc',
 '/work/ch0636/g300046/remo_results_056000/1979/e056000m197906.nc',
 '/work/ch0636/g300046/remo_results_056000/1979/e056000m197907.nc',
 '/work/ch0636/g300046/remo_results_056000/1979/e056000m197908.nc',
 '/work/ch0636/g300046/remo_results_056000/1979/e056000m197909.nc',
 '/work/ch0636/g300046/remo_results_056000/1979/e056000m197910.nc',
 '/work/ch0636/g300046/remo_results_056000/1979/e056000m197911.nc',
 '/work/ch0636/g300046/remo_results_056000/1979/e056000m197912.nc']

In [4]:
ds = open_mfdataset(filenames, chunks='auto')
ds

<xarray.Dataset>
Dimensions:                     (height10m: 1, height2m: 1, lev_4: 1, lev_5: 1, meansea: 1, nhyi: 28, nhym: 27, rlat: 433, rlon: 433, snlevs: 3, time: 12)
Coordinates:
  * rlon                        (rlon) float64 -28.93 -28.82 ... 18.49 18.6
  * rlat                        (rlat) float64 -23.93 -23.82 ... 23.49 23.6
  * meansea                     (meansea) float64 0.0
  * height10m                   (height10m) float64 10.0
  * height2m                    (height2m) float64 2.0
  * lev_4                       (lev_4) float64 1.0
  * lev_5                       (lev_5) float64 27.0
  * snlevs                      (snlevs) float64 1.0 2.0 3.0
    lon                         (rlat, rlon) float64 dask.array<chunksize=(433, 433), meta=np.ndarray>
    lat                         (rlat, rlon) float64 dask.array<chunksize=(433, 433), meta=np.ndarray>
  * time                        (time) float64 1.979e+07 1.979e+07 ... 1.979e+07
Dimensions without coordinates: nhyi, nhym
Data variables: (12/129)
    hyai                        (nhyi) float64 dask.array<chunksize=(28,), meta=np.ndarray>
    hybi                        (nhyi) float64 dask.array<chunksize=(28,), meta=np.ndarray>
    hyam                        (nhym) float64 dask.array<chunksize=(27,), meta=np.ndarray>
    hybm                        (nhym) float64 dask.array<chunksize=(27,), meta=np.ndarray>
    rotated_latitude_longitude  |S1 ...
    QDB                         (time, rlat, rlon) float32 dask.array<chunksize=(1, 433, 433), meta=np.ndarray>
    ...                          ...
    ALSOFL                      (time, rlat, rlon) float32 dask.array<chunksize=(1, 433, 433), meta=np.ndarray>
    USTRFL                      (time, rlat, rlon) float32 dask.array<chunksize=(1, 433, 433), meta=np.ndarray>
    VSTRFL                      (time, rlat, rlon) float32 dask.array<chunksize=(1, 433, 433), meta=np.ndarray>
    EVAPFL                      (time, rlat, rlon) float32 dask.array<chunksize=(1, 433, 433), meta=np.ndarray>
    TMCHFL                      (time, rlat, rlon) float32 dask.array<chunksize=(1, 433, 433), meta=np.ndarray>
    SNMLRHO                     (time, snlevs, rlat, rlon) float32 dask.array<chunksize=(1, 3, 433, 433), meta=np.ndarray>
Attributes:
    CDI:                  Climate Data Interface version 1.9.6 (http://mpimet...
    Conventions:          CF-1.6
    history:              preprocessing with pyremo = 0.1.0
    institution:          European Centre for Medium-Range Weather Forecasts
    CDO:                  Climate Data Operators version 1.9.6 (http://mpimet...
    _NCProperties:        version=2,netcdf=4.7.4,hdf5=1.10.6
    forcing_file_format:  NetCDF
    remo_version:         2.0.0
    git_branch:           nc_meta
    git_hash:             c4ee7f4
    system:               Linux eddy3 2.6.32-754.33.1.el6.x86_64 #1 SMP Mon A...

### Adjust time

In [5]:
ds['time'] = correct_monthly_time_axis(ds)
ds.time.attrs = {'units' : 'days since 1850'}
ds

<xarray.Dataset>
Dimensions:                     (height10m: 1, height2m: 1, lev_4: 1, lev_5: 1, meansea: 1, nhyi: 28, nhym: 27, rlat: 433, rlon: 433, snlevs: 3, time: 12)
Coordinates:
  * rlon                        (rlon) float64 -28.93 -28.82 ... 18.49 18.6
  * rlat                        (rlat) float64 -23.93 -23.82 ... 23.49 23.6
  * meansea                     (meansea) float64 0.0
  * height10m                   (height10m) float64 10.0
  * height2m                    (height2m) float64 2.0
  * lev_4                       (lev_4) float64 1.0
  * lev_5                       (lev_5) float64 27.0
  * snlevs                      (snlevs) float64 1.0 2.0 3.0
    lon                         (rlat, rlon) float64 dask.array<chunksize=(433, 433), meta=np.ndarray>
    lat                         (rlat, rlon) float64 dask.array<chunksize=(433, 433), meta=np.ndarray>
  * time                        (time) float64 4.713e+04 4.716e+04 ... 4.747e+04
Dimensions without coordinates: nhyi, nhym
Data variables: (12/129)
    hyai                        (nhyi) float64 dask.array<chunksize=(28,), meta=np.ndarray>
    hybi                        (nhyi) float64 dask.array<chunksize=(28,), meta=np.ndarray>
    hyam                        (nhym) float64 dask.array<chunksize=(27,), meta=np.ndarray>
    hybm                        (nhym) float64 dask.array<chunksize=(27,), meta=np.ndarray>
    rotated_latitude_longitude  |S1 ...
    QDB                         (time, rlat, rlon) float32 dask.array<chunksize=(1, 433, 433), meta=np.ndarray>
    ...                          ...
    ALSOFL                      (time, rlat, rlon) float32 dask.array<chunksize=(1, 433, 433), meta=np.ndarray>
    USTRFL                      (time, rlat, rlon) float32 dask.array<chunksize=(1, 433, 433), meta=np.ndarray>
    VSTRFL                      (time, rlat, rlon) float32 dask.array<chunksize=(1, 433, 433), meta=np.ndarray>
    EVAPFL                      (time, rlat, rlon) float32 dask.array<chunksize=(1, 433, 433), meta=np.ndarray>
    TMCHFL                      (time, rlat, rlon) float32 dask.array<chunksize=(1, 433, 433), meta=np.ndarray>
    SNMLRHO                     (time, snlevs, rlat, rlon) float32 dask.array<chunksize=(1, 3, 433, 433), meta=np.ndarray>
Attributes:
    CDI:                  Climate Data Interface version 1.9.6 (http://mpimet...
    Conventions:          CF-1.6
    history:              preprocessing with pyremo = 0.1.0
    institution:          European Centre for Medium-Range Weather Forecasts
    CDO:                  Climate Data Operators version 1.9.6 (http://mpimet...
    _NCProperties:        version=2,netcdf=4.7.4,hdf5=1.10.6
    forcing_file_format:  NetCDF
    remo_version:         2.0.0
    git_branch:           nc_meta
    git_hash:             c4ee7f4
    system:               Linux eddy3 2.6.32-754.33.1.el6.x86_64 #1 SMP Mon A...

### Select CORDEX EUR-11 domain

In [6]:
ds = select_domain(ds, 'EUR-11')
ds

<xarray.Dataset>
Dimensions:                     (height10m: 1, height2m: 1, lev_4: 1, lev_5: 1, meansea: 1, nhyi: 28, nhym: 27, rlat: 412, rlon: 424, snlevs: 3, time: 12)
Coordinates:
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.04 18.15
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.72 21.83
  * meansea                     (meansea) float64 0.0
  * height10m                   (height10m) float64 10.0
  * height2m                    (height2m) float64 2.0
  * lev_4                       (lev_4) float64 1.0
  * lev_5                       (lev_5) float64 27.0
  * snlevs                      (snlevs) float64 1.0 2.0 3.0
    lon                         (rlat, rlon) float64 dask.array<chunksize=(412, 424), meta=np.ndarray>
    lat                         (rlat, rlon) float64 dask.array<chunksize=(412, 424), meta=np.ndarray>
  * time                        (time) float64 4.713e+04 4.716e+04 ... 4.747e+04
Dimensions without coordinates: nhyi, nhym
Data variables: (12/129)
    hyai                        (nhyi) float64 dask.array<chunksize=(28,), meta=np.ndarray>
    hybi                        (nhyi) float64 dask.array<chunksize=(28,), meta=np.ndarray>
    hyam                        (nhym) float64 dask.array<chunksize=(27,), meta=np.ndarray>
    hybm                        (nhym) float64 dask.array<chunksize=(27,), meta=np.ndarray>
    rotated_latitude_longitude  |S1 ...
    QDB                         (time, rlat, rlon) float32 dask.array<chunksize=(1, 412, 424), meta=np.ndarray>
    ...                          ...
    ALSOFL                      (time, rlat, rlon) float32 dask.array<chunksize=(1, 412, 424), meta=np.ndarray>
    USTRFL                      (time, rlat, rlon) float32 dask.array<chunksize=(1, 412, 424), meta=np.ndarray>
    VSTRFL                      (time, rlat, rlon) float32 dask.array<chunksize=(1, 412, 424), meta=np.ndarray>
    EVAPFL                      (time, rlat, rlon) float32 dask.array<chunksize=(1, 412, 424), meta=np.ndarray>
    TMCHFL                      (time, rlat, rlon) float32 dask.array<chunksize=(1, 412, 424), meta=np.ndarray>
    SNMLRHO                     (time, snlevs, rlat, rlon) float32 dask.array<chunksize=(1, 3, 412, 424), meta=np.ndarray>
Attributes:
    CDI:                  Climate Data Interface version 1.9.6 (http://mpimet...
    Conventions:          CF-1.6
    history:              preprocessing with pyremo = 0.1.0
    institution:          European Centre for Medium-Range Weather Forecasts
    CDO:                  Climate Data Operators version 1.9.6 (http://mpimet...
    _NCProperties:        version=2,netcdf=4.7.4,hdf5=1.10.6
    forcing_file_format:  NetCDF
    remo_version:         2.0.0
    git_branch:           nc_meta
    git_hash:             c4ee7f4
    system:               Linux eddy3 2.6.32-754.33.1.el6.x86_64 #1 SMP Mon A...

### Get REMO code table

In [7]:
remo_table = pyremo.tables.codes.table
remo_table

,variable,description,units,layer,time_cell_method,cf_name
code,,,,,,
1,WS1,soil wetness of layer 1,m,1.0,NaN,NaN
2,WS2,soil wetness of layer 2,m,1.0,NaN,NaN
3,WS3,soil wetness of layer 3,m,1.0,NaN,NaN
4,WS4,soil wetness of layer 4,m,1.0,NaN,NaN
5,WS5,soil wetness of layer 5,m,1.0,NaN,NaN
...,...,...,...,...,...,...
491,EVAPFL,surface evaporation on lake fraction,mm,1.0,mean,NaN
492,TMCHFL,turbulent transfer coefficient of heat at the ...,NaN,1.0,mean,NaN
493,FLFRA,lake fraction,NaN,NaN,NaN,NaN


### Example Amon tas

#### Select variable and set bounds

In [8]:
var = 'tas'
var_table = remo_table[remo_table.cf_name == var]
tas_ds = create_dataset(ds, var=var, table=var_table)
tas_ds

<xarray.Dataset>
Dimensions:                     (height2m: 1, rlat: 412, rlon: 424, time: 12)
Coordinates:
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.04 18.15
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.72 21.83
  * height2m                    (height2m) float64 2.0
    lon                         (rlat, rlon) float64 dask.array<chunksize=(412, 424), meta=np.ndarray>
    lat                         (rlat, rlon) float64 dask.array<chunksize=(412, 424), meta=np.ndarray>
  * time                        (time) float64 4.713e+04 4.716e+04 ... 4.747e+04
Data variables:
    tas                         (time, height2m, rlat, rlon) float32 dask.array<chunksize=(1, 1, 412, 424), meta=np.ndarray>
    rotated_latitude_longitude  |S1 ...
Attributes:
    CDI:                  Climate Data Interface version 1.9.6 (http://mpimet...
    Conventions:          CF-1.6
    history:              preprocessing with pyremo = 0.1.0
    institution:          European Centre for Medium-Range Weather Forecasts
    CDO:                  Climate Data Operators version 1.9.6 (http://mpimet...
    _NCProperties:        version=2,netcdf=4.7.4,hdf5=1.10.6
    forcing_file_format:  NetCDF
    remo_version:         2.0.0
    git_branch:           nc_meta
    git_hash:             c4ee7f4
    system:               Linux eddy3 2.6.32-754.33.1.el6.x86_64 #1 SMP Mon A...

#### Get CORDEX CMOR test tables from github

In [10]:
import pooch

CMOR_RESOURCE = pooch.create(
    # Use the default cache folder for the OS
    path="~/.remo_cmor",
    # The remote data is on Github
    base_url="https://raw.githubusercontent.com/ludwiglierhammer/cmor-tables/main/tables/cordex-cmor-tables-test/Tables/",
    registry={
        "CORDEX_Amon.json": "51a16398ee9c1bcd8f71ee709426a3f06048c7a82f77e5dfba766b40248d4f6a",
        "CORDEX_CV.json": "83c5d19ccf5670c9850d585d89802319f52367aca109ec05221fd5df68544a97",
        "CORDEX_coordinate.json": "bf31a847cdad344b124734a5dbcb28dca740bfe496e2f85ee8af654acd213d8e",
        "CORDEX_formula_terms.json": "6f4e7c60b6089cbc873db9a2e158982b83878780ffbc8d8abe9f172c22756023",
        "CORDEX_grids.json": "970bdb5069598be9b422f9522715d97986c5d406970eb3914c177195224bdc5f",
        "CORDEX_remo_example.json": "f2434504cc9e4c438bbe5b11a1f5a63286a4c356036bad08b23ff255dccbd7d0",
    },
)

def get_cmor_tables(tables=['Amon','CV','coordinate','formula_terms','grids','remo_example'], resource=CMOR_RESOURCE):
        fmt     = "CORDEX_{}.json"
        for table in tables:
            resource.fetch(fmt.format(table))
            
get_cmor_tables()

#### Get tas values

In [12]:
tas = tas_ds.tas.values

#### Setup CMOR and load example table

In [13]:
cmor.setup(set_verbosity=cmor.CMOR_NORMAL,
           netcdf_file_action=cmor.CMOR_REPLACE)
cmor.dataset_json("cordex-cmor-tables-test/Tables/test/CORDEX_remo_example.json")

0

#### Define axes
#### The function get_bnds is just to temporary solution. Fabi will contribute his CDO solution

In [15]:
cmor.load_table("cordex-cmor-tables-test/Tables/test/CORDEX_Amon.json")

cmorTime = cmor.axis("time",
                     coord_vals  = tas_ds.time.values,
                     cell_bounds = get_bnds(tas_ds.time),
                     units       = tas_ds.time.units)
cmorLat  = cmor.axis("gridlatitude",
                     coord_vals  = tas_ds.rlat.values,
                     cell_bounds = get_bnds(tas_ds.rlat),
                     units       = tas_ds.rlat.units)
cmorLon  = cmor.axis("gridlongitude",
                    coord_vals  = tas_ds.rlon.values,
                    cell_bounds = get_bnds(tas_ds.rlon),
                    units       = tas_ds.rlon.units)

####  Calcualte new grid from cmorLat and cmorLon 
#### and define rotated_latutide_longitude; this can be read from the netcdf file directly

In [16]:
cmor.load_table("cordex-cmor-tables-test/Tables/test/CORDEX_grids.json")
cmorGRID = cmor.grid([cmorLat, cmorLon],
                     latitude=tas_ds.lat.values, 
                     longitude=tas_ds.lon.values)
rotated_latitude_longitude = {'grid_north_pole_latitude' : 39.25, 
                              'grid_north_pole_longitude' : -162, 
                              'north_pole_grid_longitude' : 0.}
cmorGM   = cmor.set_grid_mapping(cmorGRID, "rotated_latitude_longitude", 
                                 list(rotated_latitude_longitude.keys()), 
                                 list(rotated_latitude_longitude.values()), 
                                 ['','',''])

#### Now, load the Amon table again (not sure why) and adjust the tas values to the new grid

In [17]:
cmor.load_table("cordex-cmor-tables-test/Tables/test/CORDEX_Amon.json")
cmorTAS = cmor.variable("tas", tas_ds.tas.units, [cmorTime, cmorGRID])

In [18]:
#### Now wirte the file on hard on disk

In [19]:
cmor.write(cmorTAS, tas)
filename = cmor.close(cmorTAS, file_name=True)
print("Stored in:", filename)

Stored in: CORDEX/CMIP6/CORDEX/GERICS/REMO2-0-0/ECMWF/ECMWF-IFS-LR/historical/r1i1p1f1/Amon/tas/gn/v20211103/tas_Amon_REMO2-0-0_ECMWF-IFS-LR_historical_r1i1p1f1_gn_197901-197912.nc
